# Play the game

In [ ]:
# Let's make sure we have the right library
!pip install pygame

In [ ]:
# and that we are in the right kernel
import sys
print(sys.executable)

## 1) The play game alone - not user interaction
In this case, we can see the game runs, but since there is not input, it will be a score that will simply show how many times the ball went for the middle :)

In [ ]:
import os
import pygame
import random
import threading
from IPython.display import display, clear_output

def run_game():
    os.environ["SDL_AUDIODRIVER"] = "dummy"
    pygame.mixer.init()  # Safe to call, no sound will be used.

    # Initialize Pygame
    pygame.init()

    # Screen dimensions
    WIDTH, HEIGHT = 800, 600
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Goalkeeper Challenge")

    # Colors
    WHITE = (255, 255, 255)
    BLACK = (0, 0, 0)
    GREEN = (34, 139, 34)
    RED = (255, 0, 0)
    BLUE = (30, 144, 255)
    GRAY = (169, 169, 169)

    # Goalkeeper properties
    goalkeeper_width, goalkeeper_height = 50, 100
    goalkeeper_y = HEIGHT - goalkeeper_height - 50

    # Ball properties
    ball_radius = 15
    ball_positions = ["left", "middle", "right"]
    ball_x_positions = {"left": WIDTH // 4, "middle": WIDTH // 2, "right": 3 * WIDTH // 4}
    ball_y = 50
    ball_speed = 10  # Increased speed
    ball_direction = 1  # 1 for down, -1 for up

    # Font
    font = pygame.font.Font(None, 36)

    # Game variables
    clock = pygame.time.Clock()
    running = True
    score = 0
    num_rounds = 20
    current_round = 0
    ball_position = random.choice(ball_positions)
    goalkeeper_position = "middle"
    goalkeeper_x = WIDTH // 2 - (goalkeeper_width // 2)

    def draw_goal():
        pygame.draw.rect(screen, GRAY, (WIDTH // 8, HEIGHT - 150, WIDTH - WIDTH // 4, 10))  # goal post
        pygame.draw.rect(screen, GRAY, (WIDTH // 8, HEIGHT - 150, 10, 150))  # left post
        pygame.draw.rect(screen, GRAY, (WIDTH - WIDTH // 8 - 10, HEIGHT - 150, 10, 150))  # right post

    def draw_goalkeeper(x, y):
        # Draw the head
        pygame.draw.circle(screen, BLUE, (x + goalkeeper_width // 2, y - 20), goalkeeper_width // 4)
        # Draw the body
        pygame.draw.rect(screen, BLUE, (x, y, goalkeeper_width, goalkeeper_height))
        # Draw the arms
        pygame.draw.line(screen, BLUE, (x - 20, y + 20), (x + goalkeeper_width + 20, y + 20), 5)
        # Draw the legs
        pygame.draw.line(screen, BLUE, (x + 10, y + goalkeeper_height), (x - 10, y + goalkeeper_height + 40), 5)
        pygame.draw.line(screen, BLUE, (x + goalkeeper_width - 10, y + goalkeeper_height), (x + goalkeeper_width + 10, y + goalkeeper_height + 40), 5)

    def draw_field():
        pygame.draw.rect(screen, GREEN, (0, 0, WIDTH, HEIGHT))  # Green field background

    # Main game loop
    while running and current_round < num_rounds:
        draw_field()  # Draw the field background

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_LEFT:
                    if goalkeeper_position == "middle":
                        goalkeeper_position = "left"
                    elif goalkeeper_position == "right":
                        goalkeeper_position = "middle"
                    goalkeeper_x = ball_x_positions[goalkeeper_position] - (goalkeeper_width // 2)
                elif event.key == pygame.K_RIGHT:
                    if goalkeeper_position == "middle":
                        goalkeeper_position = "right"
                    elif goalkeeper_position == "left":
                        goalkeeper_position = "middle"
                    goalkeeper_x = ball_x_positions[goalkeeper_position] - (goalkeeper_width // 2)

        # Move the ball
        ball_y += ball_speed * ball_direction

        # Draw goal post
        draw_goal()

        # Draw ball
        pygame.draw.circle(screen, RED, (ball_x_positions[ball_position], ball_y), ball_radius)

        # Draw goalkeeper
        draw_goalkeeper(goalkeeper_x, goalkeeper_y)

        # Check if goalkeeper catches the ball
        if ball_direction == 1 and ball_y >= goalkeeper_y - ball_radius:
            if goalkeeper_position == ball_position:
                print("Goalkeeper caught the ball!")
                score += 1
                ball_direction = -1  # Bounce the ball back up
            else:
                print("Goalkeeper missed the ball.")
                # Reset ball for next round
                ball_position = random.choice(ball_positions)
                ball_y = 50
                ball_direction = 1
                current_round += 1
                # Reset goalkeeper to the middle
                goalkeeper_position = "middle"
                goalkeeper_x = ball_x_positions[goalkeeper_position] - (goalkeeper_width // 2)

        if ball_direction == -1 and ball_y <= ball_radius:
            # Ball has bounced back to the top, reset for next round
            ball_position = random.choice(ball_positions)
            ball_y = 50
            ball_direction = 1
            current_round += 1
            # Reset goalkeeper to the middle
            goalkeeper_position = "middle"
            goalkeeper_x = ball_x_positions[goalkeeper_position] - (goalkeeper_width // 2)

        # Display score and rounds
        score_text = font.render(f"Score: {score}", True, BLACK)
        screen.blit(score_text, (10, 10))
        round_text = font.render(f"Round: {current_round + 1}/{num_rounds}", True, BLACK)
        screen.blit(round_text, (WIDTH - 200, 10))

        pygame.display.flip()
        clock.tick(30)

    pygame.quit()
    print(f"Final Score: {score}/{num_rounds}")

# Start the game in a separate thread
game_thread = threading.Thread(target=run_game)
game_thread.start()

## 2) Play the game, with user interaction
We will capture the first moment when the ball starts the trip down, and input our movement decision, immediately after the game prints in the user did OK.
Of course, since you can see the beginning, you can always pick the right direction, but enhancing this cell can allow inputting a robot decision.

In [ ]:
import os
import pygame
import random
from IPython.display import display, Image, clear_output

# Disable sound to avoid issues in non-graphical environments
os.environ["SDL_AUDIODRIVER"] = "dummy"
pygame.mixer.init()

# Initialize Pygame
pygame.init()

# Screen dimensions
WIDTH, HEIGHT = 600, 400
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Goalkeeper Challenge")

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREEN = (34, 139, 34)
RED = (255, 0, 0)
BLUE = (30, 144, 255)
GRAY = (169, 169, 169)

# Goalkeeper properties
goalkeeper_width, goalkeeper_height = 50, 100
goalkeeper_y = HEIGHT - goalkeeper_height - 50

# Ball properties
ball_radius = 15
ball_positions = ["left", "middle", "right"]
ball_x_positions = {"left": WIDTH // 4, "middle": WIDTH // 2, "right": 3 * WIDTH // 4}
ball_y_start = 50  # Initial Y position of the ball

# Game variables
running = True
score = 0
num_rounds = 20
current_round = 0

def draw_goal():
    pygame.draw.rect(screen, GRAY, (WIDTH // 8, HEIGHT - 150, WIDTH - WIDTH // 4, 10))  # goal post
    pygame.draw.rect(screen, GRAY, (WIDTH // 8, HEIGHT - 150, 10, 150))  # left post
    pygame.draw.rect(screen, GRAY, (WIDTH - WIDTH // 8 - 10, HEIGHT - 150, 10, 150))  # right post

def draw_goalkeeper(x, y):
    # Draw the head
    pygame.draw.circle(screen, BLUE, (x + goalkeeper_width // 2, y - 20), goalkeeper_width // 4)
    # Draw the body
    pygame.draw.rect(screen, BLUE, (x, y, goalkeeper_width, goalkeeper_height))
    # Draw the arms
    pygame.draw.line(screen, BLUE, (x - 20, y + 20), (x + goalkeeper_width + 20, y + 20), 5)
    # Draw the legs
    pygame.draw.line(screen, BLUE, (x + 10, y + goalkeeper_height), (x - 10, y + goalkeeper_height + 40), 5)
    pygame.draw.line(screen, BLUE, (x + goalkeeper_width - 10, y + goalkeeper_height), (x + goalkeeper_width + 10, y + goalkeeper_height + 40), 5)

def draw_field():
    pygame.draw.rect(screen, GREEN, (0, 0, WIDTH, HEIGHT))  # Green field background

def save_screenshot(round_number):
    filename = f"round_{round_number + 1}.png"
    pygame.image.save(screen, filename)
    return filename

def run_game():
    global running, current_round, score

    while running and current_round < num_rounds:
        ball_position = random.choice(ball_positions)
        goalkeeper_position = "middle"
        goalkeeper_x = ball_x_positions[goalkeeper_position] - (goalkeeper_width // 2)
        ball_y = ball_y_start  # Initialize the ball's Y position for each round

        # Draw the initial state of the game
        draw_field()
        draw_goal()
        draw_goalkeeper(goalkeeper_x, goalkeeper_y)
        pygame.draw.circle(screen, RED, (ball_x_positions[ball_position], ball_y), ball_radius)
        pygame.display.flip()
        
        # Save a screenshot at the start of the round and display it
        screenshot_path = save_screenshot(current_round)
        clear_output(wait=True)
        display(Image(filename=screenshot_path))
        
        print(f"\nRound {current_round + 1}/{num_rounds}")
        print(f"Current score: {score}")
        
        # Player inputs their move
        player_input = input("Move the goalkeeper (left, middle, right): ").strip().lower()
        
        if player_input not in ["left", "middle", "right"]:
            print("Invalid input! Please type 'left', 'middle', or 'right'.")
            continue
        
        # Check if goalkeeper catches the ball
        if player_input == ball_position:
            print(f"Goalkeeper moved to {player_input} and caught the ball!")
            score += 1
        else:
            print(f"Goalkeeper moved to {player_input}, but missed the ball! The ball was at {ball_position}.")
        
        # Move to the next round
        current_round += 1
        
        if current_round >= num_rounds:
            running = False

    print(f"\nFinal Score: {score}/{num_rounds}")
    pygame.quit()

# Run the game loop in the current thread
run_game()

### Let's now clean the images:

In [1]:
import os
import glob

# Find all PNG files that match the pattern "round_*.png"
png_files = glob.glob("round_*.png")

# Loop through and remove each file
for file in png_files:
    os.remove(file)
    print(f"Deleted {file}")

print("All round screenshots have been removed.")


Deleted round_1.png
Deleted round_10.png
Deleted round_11.png
Deleted round_12.png
Deleted round_13.png
Deleted round_14.png
Deleted round_15.png
Deleted round_16.png
Deleted round_17.png
Deleted round_18.png
Deleted round_19.png
Deleted round_2.png
Deleted round_20.png
Deleted round_3.png
Deleted round_4.png
Deleted round_5.png
Deleted round_6.png
Deleted round_7.png
Deleted round_8.png
Deleted round_9.png
All round screenshots have been removed.


## - end -